In [1]:
import warnings
warnings.filterwarnings('ignore')
import main

In [9]:
search_engine = main.SearchEngine(r'C:\Users\sergiojesus\Desktop\Document_Clustering\CV', lang='pt')

In [15]:
%time results = search_engine.run_search(['C++', 'java'], num_returns=10, summary=True)

Results:


,File,Link,Score,Summary,Results
0,Rui Moreira,CV\10-outubro 2016\Rui Moreira.pdf,1,▪ Implementação de projeto técnico certificado de telecomunicações em edifícios de acordo com a regulamentação técnica em vigor da ANACOM e recomendações municipais Responsável por angariação e orientação de formação em contexto de trabalho em empresas do ramo Sector de atividade: Ensino / Formação técnico profissional nas áreas de profissional nas áreas de,"▪ Programação em diversas plataformas (Omnis, Java em diversas plataformas (Omnis, Java J2SE, C++, C#, .Net, JavaScript, HTML, PHP, CSS, Flash) ▪ Programação de Computadores ( VBasic, C++, JAVA, SQL server, PHP, ▪ Linguagens de programação: PHP, JAVA J2SE, JAVASCRIPT, .NET, VBASIC, ▪ Formação de programação em V. BASIC, C++, bases de dados mySQL e V. BASIC, C++, bases de dados mySQL e Tigerlogic Omnis, C++, C#, ASSEMBLY, HTML;"
1,Jorge Ferrão CV,CV\geral_2018\Jorge Ferrão CV.pdf,0.811577,"Tenho conhecimentos de programação em C/C++, Java, C#, Python e Javascript, e experiencia com computação gráfica, visão por computador, desenvolvimento de aplicações multiplataforma, programação de microcontroladores, desenvolvimento de jogos, desenvolvimento de sistemas de informação, aprendizagem automática e desenvolvimento web","Framework para criação de animação vetorial em Java. Java, LibGDX, JNI, C++ Jogo de damas (2014) Jogo de damas humano vs computador, com realidade aumentada. Desenvolvimento de sistema de visão para deteção de tabuleiros de damas. OpenCV, C++ LibGDX, Java Software developer C++ / ROS para o robô autónomo WiiGo Algoritmos localização e navegação autónoma Ferramentas para visualização de dados Página do projeto disponível em www.nunustudio.org Versão online disponível em http://www.nunustudio.org/editor/editor three.js, WebGL, javascript, node.js Biblioteca para renderização de gráficos 3D no browser com WebGL. WebGL, javascript e GLSL Pagina projeto GitHub github.com/tentone/IDKwGL Tenho conhecimentos de programação em C/C++, Java, C#, Python e Javascript, e experiencia com computação gráfica, visão por computador, desenvolvimento de aplicações multiplataforma, programação de microcontroladores, desenvolvimento de jogos, desenvolvimento de sistemas de informação, aprendizagem automática e desenvolvimento web"
2,José Manuel Miranda Ferrão_12-2017,CV\Dezembro 2017\José Manuel Miranda Ferrão_12-2017.pdf,0.811577,"Tenho conhecimentos de programação em C/C++, Java, C#, Python e Javascript, e experiencia com computação gráfica, visão por computador, desenvolvimento de aplicações multiplataforma, programação de microcontroladores, desenvolvimento de jogos, desenvolvimento de sistemas de informação, aprendizagem automática e desenvolvimento web","Framework para criação de animação vetorial em Java. Java, LibGDX, JNI, C++ Jogo de damas (2014) Jogo de damas humano vs computador, com realidade aumentada. Desenvolvimento de sistema de visão para deteção de tabuleiros de damas. OpenCV, C++ LibGDX, Java Software developer C++ / ROS para o robô autónomo WiiGo Algoritmos localização e navegação autónoma Ferramentas para visualização de dados Página do projeto disponível em www.nunustudio.org Versão online disponível em http://www.nunustudio.org/editor/editor three.js, WebGL, javascript, node.js Biblioteca para renderização de gráficos 3D no browser com WebGL. WebGL, javascript e GLSL Pagina projeto GitHub github.com/tentone/IDKwGL Tenho conhecimentos de programação em C/C++, Java, C#, Python e Javascript, e experiencia com computação gráfica, visão por computador, desenvolvimento de aplicações multiplataforma, programação de microcontroladores, desenvolvimento de jogos, desenvolvimento de sistemas de informação, aprendizagem automática e desenvolvimento web"
3,Miguel Ribeiro_07_2017,CV\IT\Miguel Ribeiro_07_2017.pdf,0.803458,Experiência nas linguagens de programação e tecnologias:  Back end: Java; Spring Framework; Apache Maven;  Mobile: Apache Cordova; Ionic Framework; TypeScript; Tecnologias

Wall time: 222 ms


In [2]:
search_engine_2 = main.SearchEngineElasticSearch(init_path=r'C:\Users\sergiojesus\Desktop\Document_Clustering\CV', index_name='cv_sergio')

In [6]:
% time results = search_engine_2.query_database('design java C#', summary=True, max_size=10)

Results:


,File,Link,Score,Summary,Results
0,Cem Uyar_08_2017,cv\IT\Cem Uyar_08_2017.pdf,17.2184,"+ Developed back end services for internet banking applications with C#, ASP.NET web pages for internet banking operations and C# console applications. + Developed back end services for banking applications with Java and Other Used Technologies: ASP.NET, CSS, JavaScript, jQuery, HTML Used Technologies: ASP.NET MVC, CSS, JavaScript, jQuery, HTML, SQL, C#","+ Developed back end services for internet banking applications with C#, Java applications for date scheduled operations of the Bank. + Java ASP.NET web pages for internet banking operations and C# console applications. + Practiced and learned object oriented languages: Java, C# + Designed pages for Bank's branch operations with Java AURORA. + Developed back end services for banking applications with Java and + C# Other Used Technologies: ASP.NET, CSS, JavaScript, jQuery, HTML + Developed prototype of Snap Chat in C# with using Scrum methodology. Implemented 10 different design patterns to an already existed C# project. + Designed database scheme. Used Technologies: ASP.NET MVC, CSS, JavaScript, jQuery, HTML, SQL, C# + JavaScript. jQuery, ReactJS + Practiced and learned algorithm designing with C language."
1,Tiago Martins_12-2017,cv\Dezembro 2017\Tiago Martins_12-2017.pdf,16.6012,Software Development Web and others Computer Networks Multimedia Design Database programer Software Development Web and others Computer Networks Multimedia Design Database programer Application analyst Software Development Web and others Software Engineering Computer Networks (similar to Cisco CCNP (routing)) Systems Administrator Multimedia Design Database programer Application analyst,"Software Development Web and others Computer Networks Multimedia Design Database programer Application analyst JAVA programming Creation of all HTML and Designing the CSS Company: Preggo Menina Design Group Software Development Web and others Computer Networks Multimedia Design Database programer Software Development Web and others Software Engineering Computer Networks (similar to Cisco CCNP (routing)) Systems Administrator Multimedia Design Database programer Application analyst Designing adjustments on CSS Designing Web app on Adobe Illustrator JAVA JAVA ∎ C# ∎ VB ∎ HTML ∎ CSS ∎ SQL At Preggo, Menina Design’s productive section, I was responsible for the following:"
2,Carlos Pedro Monaco Tullio Muniz_12-2017,cv\Dezembro 2017\Carlos Pedro Monaco Tullio Muniz_12-2017.pdf,16.5729,"The main responsibility was to specify the software architecture and how it should be implemented by the development team. My specialties:ASP.NET MVC, Entity Framework, NHibernate, StructureMAP, C #, Development together with the software house new portlets to integrate the portal with important services to the society. Development of software solutions for several engineering disciplines integration problem.","Java Software quality and testing with Java Languages & Technologies ( Java, C#, VB.NET, Delphi, Python, Android, Xamarin) Foxconn Technology Group is the leading provider of design, development, manufacturing, assembly and after sales services to global leaders in computers, communication and entertainment Software Design (Functional; OOP; Patterns) JAVA JAVA, Unity Tests (VS, NUnit and Unit), Integrated Tests (VS, NUnit and Unit),"
3,Nitin Sheshmal Jain_12-2017,cv\Dezembro 2017\Nitin Sheshmal Jain_12-2017.pdf,16.3057,"Role: Analysis, Design and Develop dynamic web application. Role: Analysis, Design and Develop dynamic Payment gateway integration for E commerce application.","Tools and Technologies Used: ASP.Net 3.5, C#, JavaScript, DotNetNuke 5.X. 1. C# Combo Role: Analysis, Design and Develop dynamic web application. Project Name: Exam Designer Description: Development of Exam Designer Description: JasperReport is a tool which creates multi format document (PDF, XLS, HTML, CSV, and many more) on the fly. JasperReport is a java reporting tool.

Wall time: 286 ms
